In [63]:
import requests
from datetime import datetime
import pandas as pd

In [ ]:
api_key = '1212688ede774309ad6e24ee2a5bd970'

date = '2024-12-01'  # Date au format YYYY-MM-DD

def get_news(date, api_key):
    # URL de NewsAPI
    url = f'https://newsapi.org/v2/everything?q=adobe&from={date}&sortBy=publishedAt&apiKey={api_key}'

    # Faire la requête
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        
        news_data = []

        # Vérifier s'il y a des articles dans la réponse
        if data['articles']:
            df = pd.json_normalize(data['articles'])[["publishedAt", "title", "description"]]
            
            df.columns = ["Date", "News", "Description"]
            df["News"] = df["News"] + df["Description"]
            df = df[["Date", "News"]]
            return df
        else:
            print("Aucun article trouvé pour cette date.")
            return pd.DataFrame() 
    else:
        print(f"Erreur lors de la récupération des actualités. Code : {response.status_code}")
        return pd.DataFrame()

In [60]:
df = get_news(date, api_key)

In [61]:
df.shape

(100, 2)

In [ ]:
# https://fred.stlouisfed.org/series/EMVMACROBUS
# https://fred.stlouisfed.org/series/CPIAUCSL
# https://fred.stlouisfed.org/series/EXPINF1YR
# https://fred.stlouisfed.org/series/LNS12032195
# https://fred.stlouisfed.org/series/UMCSENT

In [70]:
files = [
    "EMVMACROBUS.csv",
    "CPIAUCSL.csv",
    "EXPINF1YR.csv",
    "LNS12032195.csv",
    "UMCSENT.csv"
]

combined_df = None

for file in files:
    df = pd.read_csv('extra_data/' + file)
    if combined_df is None:
        combined_df = df  # Premier fichier
    else:
        combined_df = pd.merge(combined_df, df, on="DATE", how="outer") 

In [72]:
start_date = "2009-01-01"
end_date = "2024-10-31"

combined_df['DATE'] = pd.to_datetime(combined_df['DATE'])

# Filtrer les dates dans l'intervalle
filtered_df = combined_df[(combined_df['DATE'] >= start_date) & (combined_df['DATE'] <= end_date)]

filtered_df

,DATE,EMVMACROBUS,CPIAUCSL,EXPINF1YR,LNS12032195,UMCSENT
744,2009-01-01,0.17876,211.933,0.473575,6019.0,61.2
745,2009-02-01,0.19250,212.705,0.381647,6602.0,56.3
746,2009-03-01,0.17958,212.495,-0.480678,6972.0,57.3
747,2009-04-01,0.27625,212.709,1.228936,6755.0,65.1
748,2009-05-01,0.11837,213.022,1.297246,6838.0,68.7
...,...,...,...,...,...,...
929,2024-06-01,0.00000,313.049,2.725840,2832.0,68.2
930,2024-07-01,0.00000,313.534,2.555644,2985.0,66.4
931,2024-08-01,0.31524,314.121,2.347059,3303.0,67.9
932,2024-09-01,0.44263,314.686,2.236121,2999.0,70.1


In [73]:
missing_rate = filtered_df.isnull().mean() * 100

print("Taux de valeurs manquantes par colonne (%):")
print(missing_rate)

Taux de valeurs manquantes par colonne (%):
DATE           0.0
EMVMACROBUS    0.0
CPIAUCSL       0.0
EXPINF1YR      0.0
LNS12032195    0.0
UMCSENT        0.0
dtype: float64


In [74]:
combined_df.to_csv("macro_sentiment_info.csv", index=False)